Consider a sky-diver falling towards the surface of Earth. The diver brought with them a temperature sensor that took multiple measurements along the descent. The diver is interested in determining their terminal velocity given these measurements.

Problem: Given a temperature measurement, can one estimate the velocity as a function of time?

In [411]:
using LinearAlgebra
using Random
using PlotlyJS

include("kalmanflt.jl");

The system is a particle falling due to gravity WITH air resistance. For this simulation, the falling object will acheive terminal velocity. The velocity as a function of time is modeled as

$$
v(t) = v_{\infty} \tanh\left(\frac{g t}{v_{\infty}}\right)
$$

where $v_{\infty}$ is the terminal velocity as time approaches infinity and $g$ is the acceleration due to gravity. 

In [412]:
accel_ms2 = -9.8 # meter / sec^2
v_inf = 54 # meter / sec
time_steps = 100
delta_time_sec = 0.15  # sec
time_axis_sec = [delta_time_sec * (t-1) for t in 1:time_steps];

Plot the velocity function

In [413]:
plot(
    [
        # Velocity
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[v_inf * tanh(accel_ms2 * t / v_inf) for t in time_axis_sec],
            name="Velocity"
        ),
    ],
    Layout(
        title="Velocity Model",
        xaxis_title="Time (second)",
        yaxis_title="Velocity (meter / second)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

We want to model the position and velocity of an object falling, but we CANNOT use the following equations
$$
\begin{align}
h &= h_{0} + v_0 t + \frac{1}{2} a t^{2} \\
v &= v_{0} + g t
\end{align}
$$

since the acceleration is NOT constant in time

$$
\begin{align}
a &= \frac{dv}{dt} = g \ \text{sech}^2\left(\frac{g t}{v_{\infty}}\right)
\end{align}
$$

Let's consider the following model instead

$$
\begin{align}
h \rightarrow h + \delta h &= \sum\limits_{n=0}^{\infty}\left.\left(\frac{d}{dt}\right)^{n} h\right|_{t=T}\frac{\left(\Delta t\right)^n}{n!} \\
     &= h(T) + v(T) \Delta t + \frac{1}{2} a(T) \left(\Delta t\right)^2 + \frac{1}{3!} j(T) \left(\Delta t\right)^3 + \frac{1}{4!} s(T) \left(\Delta t\right)^4 + \mathcal{O}(t^5)
\end{align}
$$

The terms in the sum above are given by

$$
\begin{align}
v(t) &= v_{\infty} \tanh\left(\frac{g t}{v_{\infty}}\right) & \text{Velocity}\\
a(t) &= \frac{dv}{dt}     = g \ \text{sech}^2\left(\frac{g t}{v_{\infty}}\right) & \text{Acceleration}\\
j(t) &= \frac{d^2v}{dt^2} = -\frac{2 g^2}{v_{\infty}} \tanh \left(\frac{g t}{v_{\infty}}\right) \text{sech}^2\left(\frac{g t}{v_{\infty}}\right) & \text{Jerk} \\
s(t) &= \frac{d^3v}{dt^3} = \frac{2 g^3}{v_{\infty}^2} \left(\cosh \left(\frac{2 g t}{v_{\infty}}\right)-2\right) \text{sech}^4\left(\frac{g t}{v_{\infty}}\right) & \text{Snap} \\
\vdots
\end{align}
$$

and we must determine when to truncate the series. Let's plot each derivative.

Plot each successive term in the sum

In [414]:
plot(
    [
        # Velocity
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[v_inf * tanh(accel_ms2 * t / v_inf) for t in time_axis_sec],
            name="Velocity"
        ),
        # Acceleration
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[accel_ms2 * sech(accel_ms2 * t / v_inf)^2 for t in time_axis_sec],
            name="Acceleration"
        ),
        # Jerk
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[-2 * accel_ms2^2 / v_inf * tanh(accel_ms2 * t / v_inf)^2 * sech(accel_ms2 * t / v_inf)^2 for t in time_axis_sec],
            name="Jerk"
        ),
        # Snap
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[2 * accel_ms2^3 / v_inf^2 * (cosh(2 * accel_ms2 * t / v_inf) - 2) * sech(accel_ms2 * t / v_inf)^4 for t in time_axis_sec],
            name="Snap"
        ),
    ],
    Layout(
        title="Velocity Model",
        xaxis_title="Time (second)",
        yaxis_title="(d/dt)^n v per Unit Time",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

For this example, we will truncate the series up to the `j` jerk term.

What we need now is a mapping between temperature and altitude.
I will use the data from https://www.engineeringtoolbox.com/air-altitude-temperature-d_461.html and linearize the temperature model. For the terminal velocity, I make-up a weak, linear relationship between temperature and terminal velocity.

In [415]:
"""
    get_velocity_infinity_ms(temperature_K)

Get the velocity using a linear temperature model. The return values are between ~46 and ~54 meters / sec for most terrestial temperatures

# Arguments
- `temperature_K::Number`: The temperature in Kelvin

# Returns
`Float64` of the estimated terminal velocity in meters / second
"""
function get_velocity_infinity_ms(
        temperature_K::Number
    )::Float64
    return -0.1481 * temperature_K + 94.44
end

"""
    get_temperature_K(altitude_m[, vary_slope_by, vary_intercept_by])

Get the temperature given an atltitude above the Earth's surface which can be varied with the last two optional parameters

# Arguments
- `altitude_m::Number`: The altitude above Earth in meters

# Optional Arguments
- `vary_slope_by::Number`: Positive definite constant to vary the slope of temperature changes
- `vary_intercept_by::Number`: Positive definite constant to vary the intercept of temperature changes

# Returns
`Float64` of the estimated temperature Kelvin
"""
function get_temperature_K(
        altitude_m::Number, 
        vary_slope_by::Number=0.0, 
        vary_intercept_by::Number=0.0
    )::Float64
    slope = -0.006_835 # Kelvin / meter
    intercept = 288.706 # Kelvin
    if vary_slope_by > 0
        slope *= vary_slope_by
    end
    if vary_intercept_by > 0
        intercept *= vary_intercept_by
    end
    temerature = slope * altitude_m + intercept
    return temerature
end;

Let's plot these two models

In [416]:
temperature_range_K = 273.0:300.0

plot(
    [
        scatter(
            mode="markers+lines",
            x=temperature_range_K,
            y=[get_velocity_infinity_ms(T) for T in temperature_range_K]
        )
    ],
    Layout(
        title="Terminal Velocity Model",
        xaxis_title="Temperature (Kelvin)",
        yaxis_title="Velocity (meters / seconds)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

In [417]:
altitude_range_m = 0.0:100.0:3000.0

plot(
    [
        scatter(
            mode="markers+lines",
            x=altitude_range_m,
            y=[get_temperature_K(A) for A in altitude_range_m]
        )
    ],
    Layout(
        title="Temperature Model",
        xaxis_title="Altitude (meters)",
        yaxis_title="Temperature (Kelvin)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

With those two models established, let's define the observation matrix as

$$
\begin{align}
H &= H(t) \\
&=
\begin{bmatrix}
T(a)/a \\
0
\end{bmatrix}
\end{align}
$$

where $a$ is the altitude and $T$ is the temperature as a function of $a$

In [418]:
"""
    get_observation_mat(altitude_m[, vary_slope_by, vary_intercept_by])

Get the observations matrix given an atltitude above the Earth's surface which can be varied with the last two optional parameters

# Arguments
- `altitude_m::Number`: The altitude above Earth in meters

# Optional Arguments
- `vary_slope_by::Number`: Positive definite constant to vary the slope of temperature changes
- `vary_intercept_by::Number`: Positive definite constant to vary the intercept of temperature changes

# Returns
`Float64` of the observations matrix
"""
function get_observation_mat(
        altitude_m::Number,
        vary_temperature_slope_by::Number=0.0,
        vary_temperature_intercept_by::Number=0.0
    )::Matrix{Float64}
    temperature_K = get_temperature_K(
        altitude_m, 
        vary_temperature_slope_by, 
        vary_temperature_intercept_by
    )
    velocity_terminal_ms = get_velocity_infinity_ms(temperature_K)
    observation_mat = [temperature_K/altitude_m 0.0]
    return observation_mat
end;

Let's consider the state transition and control matrices. They are given by
$$
\begin{align}
x_n =& F x_{n-1} + G u \\
    =& 
\begin{bmatrix}
1 & \Delta t \\
0 & 1
\end{bmatrix}
\begin{bmatrix}
h_{n-1} \\
v_{n-1}
\end{bmatrix}
+
\begin{bmatrix}
\frac{1}{2} \left(\Delta t\right)^2 & \frac{1}{3!} \left(\Delta t\right)^3 \\
\Delta t & \frac{1}{2} \left(\Delta t\right)^2
\end{bmatrix}
\begin{bmatrix}
a(T) \\
j(T)
\end{bmatrix} \\
\begin{bmatrix}
h_n \\
v_n
\end{bmatrix}
=& 
\begin{bmatrix}
h(t_{n-1}) + v(t_{n-1}) \Delta t + \frac{1}{2} a(t_n) \left(\Delta t\right)^2 + \frac{1}{3!} j(t_{n-1}) \left(\Delta t\right)^3 \\
v(t_{n-1}) + a(t_{n-1}) \Delta t + \frac{1}{2} j(t_{n-1}) \left(\Delta t\right)^2
\end{bmatrix}
\end{align}
$$

For ease of coding, I chose to write $Gu$ instead as

$$
Gu = 
\begin{bmatrix}
\frac{1}{2} a(T) \left(\Delta t\right)^2  + \frac{1}{3!} j(T) \left(\Delta t\right)^3 & 0\\
0 & a(T) \Delta t + \frac{1}{2} j(T) \left(\Delta t\right)^2
\end{bmatrix}
\begin{bmatrix}
1 \\
1
\end{bmatrix}
$$

In [419]:
"""
    get_control_mat(state_vec, total_time_sec, delta_time_sec)

Get the control matrix

# Arguments
-`state_vec` Must be a 2-item vector exactly as `Vector{Float64}([p, v])` where `p` is the position and `v` is the velocity
-`total_time_sec` Total time of system measurements
-`delta_time_sec:`Time differential between measurements

# Returns
`Matrix{Float64}` of the control matrix
"""
function get_control_mat(
        state_vec::Vector,
        total_time_sec::Number,
        delta_time_sec::Number
    )::Matrix{Float64}
    temperature_K = get_temperature_K(state_vec[1])
    vel_inf_ms = get_velocity_infinity_ms(temperature_K)
    arg = accel_ms2 * total_time_sec / vel_inf_ms
    current_acceleration_ms2 = accel_ms2 * sech(arg)^2
    current_jerk_ms3 = -2 * accel_ms2^2 / vel_inf_ms * sech(arg)^2 * tanh(arg)
    control_mat11 = (1. / factorial(2) * current_acceleration_ms2 * delta_time_sec^2
                     + 1. / factorial(3) * current_jerk_ms3 * delta_time_sec^3)
    control_mat_22 = (current_acceleration_ms2 * delta_time_sec
                       + 1. / factorial(2) * current_jerk_ms3 * delta_time_sec^2)
    control_mat = Diagonal([control_mat11, control_mat_22])
    return control_mat
end;

In [420]:
"""
    get_model_estimate(state_vec, total_time_sec, delta_time_sec)

Get the model estimate for the given state vector.

# Arguments
-`state_vec` Must be a 2-item vector exactly as `Vector{Float64}([p, v])` where `p` is the position and `v` is the velocity
-`total_time_sec` Total time of system measurements
-`delta_time_sec:`Time differential between measurements

# Returns
`Vector{Float64}` of the state estimate
"""
function get_model_estimate(
        state_vec::Vector,
        total_time_sec::Number,
        delta_time_sec::Number
    )::Vector{Float64}
    """Get the model estimate for current state

    :param state_vec: State vector
    :param total_time_sec: Total time of system measurements
    :param delta_time_sec: Time differential between measurements
    :returns: Vector
    """

    state_transition = [1.0 delta_time_sec; 0.0 1.0]
    control_mat = get_control_mat(state_vec, total_time_sec, delta_time_sec)
    external_input_vector = Vector([1, 1])
    model_estimate_vec = (state_transition * state_vec
                          + control_mat * external_input_vector)
    return model_estimate_vec
end

"""
    get_model_estimate(true_state, total_time_sec, delta_time_sec, artificial_noise_K)

Get the simulated measurement as a function of the true state

# Arguments
 -`true_state`: The true state vector which must be a 2-item vector exactly as `Vector{Float64}([p, v])` where `p` is the position and `v` is the velocity
 -`total_time_sec`: Total time of system measurements
 -`delta_time_sec`: Time differential between measurements
 -`artificial_noise_K`: Artificial noise magnitude of temperatrue measurement 

# Returns
`Vector{Float64}` of the model estimate
"""
function get_measurement_vec(
        true_state::Vector,
        vary_temperature_slope_by::Float64=0.0,
        vary_temperature_intercept_by::Float64=0.0,
        artificial_noise_K::Float64=0.1
    )::Vector{Float64}
    """In a 'real' physical system, we would not need to simulate the measurement"""
    altitude_m = true_state[1]
    observation_mat = get_observation_mat(
        altitude_m, 
        vary_temperature_slope_by, 
        vary_temperature_intercept_by
    )

    measurement_vec::Vector = observation_mat * true_state + Vector([artificial_noise_K])
    return measurement_vec

end;

In [421]:
"""Use these two variables to simulate differences variations between the truth and observation
for the temperature vs altitude
"""
temperature_variation_slope = 0.0
temperature_variation_intercept = 0.0

"""Temperature variations"""
artificial_noise_K = 0.1  # Kelvin
observation_cov = 5.0^2 * I(1)  # Kelvin^2

"""Define a system where a falling object is thought to be released at rest 
at 3 km above the Earth's surface with wind resistence. The true position 
is 3020.
"""
true_state_vector = Vector([3020.0, +0.05])
initial_state_vector = Vector([3000.0, -0.001])
initial_state_cov = [100.0^2 10.0; 10.0 1.0^2]  # Diagonal([100.0^2, 1.0^2])
initial_state = kfstate{Float64}(initial_state_vector, initial_state_cov)

"""Initialize the transition and control matrices at first time step"""
state_transition_mat = [1.0 delta_time_sec; 0.0 1.0]
control_mat = get_control_mat(initial_state_vector, delta_time_sec, delta_time_sec)
system_noise_mat = Diagonal([10.0^2, 1.0^2])
external_input_vector = Vector([1, 1])
system = kfsystem{Float64}(state_transition_mat, control_mat, system_noise_mat)

"""Initialize the observation matrix"""
observation_mat = get_observation_mat(
    initial_state.state[1],
    temperature_variation_slope,
    temperature_variation_intercept)
observations = kfobservation{Float64}(observation_mat, observation_cov);

In [422]:
next_state = kfstate{Float64}(initial_state.state, initial_state.cov)
states_fcn_time = Vector{kfstate{Float64}}(undef, time_steps)
true_state_fcn_time = Array{Vector{Float64}}(undef, time_steps)
measurements_array = Array{Float64}(undef, time_steps);

In [423]:
for step in 1:time_steps

    # At current time, get measurement
    time_sec = (step - 1) * delta_time_sec
    true_state_vector = get_model_estimate(true_state_vector, time_sec, delta_time_sec)
    measurement_vec = get_measurement_vec(
        true_state_vector,
        temperature_variation_slope,
        temperature_variation_intercept,
        artificial_noise_K
    )
    true_state_fcn_time[step] = true_state_vector
    
    # Predict the next state
    predicted_state = extrapulate_state(next_state, system, external_input_vector)

    # Update the observation matrix estimate
    new_observations_mat = get_observation_mat(
        predicted_state.state[1],
        temperature_variation_slope,
        temperature_variation_intercept
    )
    observations.obs = new_observations_mat
    
    # Correct/update the state prediction
    next_state = update_state(predicted_state, observations, measurement_vec)

    # Update the system control and observation matrix for the next measurement
    new_control_mat = get_control_mat(next_state.state, time_sec, delta_time_sec)
    system.control = new_control_mat

    # Save the results
    measurements_array[step] = measurement_vec[1]
    states_fcn_time[step] = next_state

end

In [424]:
plot(
    [
        # True position
        scatter(
            mode="markers",
            x=time_axis_sec,
            y=measurements_array,
            name="Measurement"
        )
    ],
    Layout(
        title="Measurement",
        xaxis_title="Time (second)",
        yaxis_title="Temperature (Kelvin)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

In [425]:
plot(
    [
        # True position
        scatter(
            mode="markers",
            x=time_axis_sec,
            y=[tr_st[1] for tr_st in true_state_fcn_time],
            name="True Position"
        ),
        # Estimated position
        scatter(
            mode="markers",
            x=time_axis_sec,
            y=[s.state[1] for s in states_fcn_time],
            error_y=attr(
                type="data",
                array=[sqrt(s.cov[1, 1]) for s in states_fcn_time]
            ),
            name="Estimated Position"
        )
    ],
    Layout(
        title="Position Measurement",
        xaxis_title="Time (second)",
        yaxis_title="Position (meters)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields error_y, mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

In [426]:
plot(
    [
        # True velocity
        scatter(
            mode="markers",
            x=time_axis_sec,
            y=[tr_st[2] for tr_st in true_state_fcn_time],
            name="True Velocity"
        ),
        # Estimated velocity
        scatter(
            mode="markers",
            x=time_axis_sec,
            y=[s.state[2] for s in states_fcn_time],
            error_y=attr(
                type="data",
                array=[sqrt(s.cov[2, 2]) for s in states_fcn_time]
            ),
            name="Estimated Velocity"
        )
    ],
    Layout(
        title="Velocity Measurement",
        xaxis_title="Time (second)",
        yaxis_title="Velocity (meters / seconds)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields error_y, mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"